This is not a writeup that I am happy with. I will keep it on my GitHub but not publish it on CTFtime.


In the Challenge we are given a short python file with one function `f(n)` and one class `G`. 

Function `f` is fairly simple as it just generates a list of primes from 1 until n=1,000,000.

In [1]:
def f(n):
    q=[True]*(n + 1)
    r=2
    while r**2<=n:
        if q[r]:
            for i in range(r**2,n+1,r):q[i] = False
        r += 1
    return [p for p in range(2,n+1) if q[p]]

When we create an instance of the class `G`, we initialise it with the list of primes as `self.f` and a state `self.state=1` which will be our encrypted flag.

The class `G` also contains a function `move` which iterates over the list of primes until it finds a prime such that `self.state%p!=0`. 

In [2]:
class G:
    def __init__(self, f):
        self.f = f
        self.state = 1
    def move(self):
        q=1
        for p in self.f:
            if self.state%p!=0:
                self.state=self.state*p//q
                return
            q*=p

Using `G`, the flag is encrypted by converting the flag into a long $f$ and then calling `G.move` $f$ times. The resulting state is the encrypted flag.

We are also given the encrypted flag in the file

In [3]:
from Cryptodome.Util.number import bytes_to_long

#flag = open('flag.txt','r').read().strip().encode()
flag = 'fla'.encode()
flag=bytes_to_long(flag)
primes = f(pow(10,6))
gen = G(primes)
for _ in range(flag):gen.move()
print('enc =',gen.state)
# enc = 31101348141812078335833805605789286074261282187811930228543150731391596197753398457711668323158766354340973336627910072170464704090430596544129356812212375629361633100544710283538309695623654512578122336072914796577236081667423970014267246553110800667267853616970529812738203125516169205531952973978205310

enc = 27457552467146386


Note that the `move` function is independent of the states and the encrypted flag only depends on the number of times that `g.move()` is called. We can start to solve this challenge by investigating the first states. 

We can see that the state will start with a prime number $p$ (e.g. 5) which is followed by multiples of $p$. Eventually, the state will become the next larger prime. 

|State |Max primefactor|
|--- |--- |
|**2**|2|
|**3**|3|
|6|3|
|**5**|5|
|10|5|
|15|5|
|30|5|
|**7**|7|
|14|7|
|21|7|
|42|7|
|35|7|
|70|7|
|105|7|
|210|7|
|**11**|11|
|22|11|
|33|11|
|66|11|
|55|11|
|110|11|
|165|11|
|330|11|
|77|11|
|154|11|
|231|11|
|462|11|
|385|11|
|770|11|
|1155|11|
|2310|11|
|**13**|13|

If we count the states for the primefactory, we find that we have $2^i$ states for the $i^{th}$ primefactor 

|Index|Prime | Frequency |
|--- |--- |--- |
|0|2|1|
|1|3|2|
|2|5|4|
|3|7|8|
|4|11|16|
|5|13|32|
|6|13|64|
|$\vdots$|$\vdots$|$\vdots$|


This means that if our encrypted flag consists of the primefactors $2\cdot7\cdot11=154$ with respective indices (0,3,4). With the indices, we can calculate the original flag $2^0\cdot2^3\cdot2^4=25$

We can validate this with the original script:


In [4]:
flag=25
primes = f(pow(10,6))
gen = G(primes)
for _ in range(flag):gen.move()
print('enc =',gen.state)

enc = 154


Finally, we can decrypt the encoded flag:

In [5]:
from sympy.ntheory import factorint

enc = 31101348141812078335833805605789286074261282187811930228543150731391596197753398457711668323158766354340973336627910072170464704090430596544129356812212375629361633100544710283538309695623654512578122336072914796577236081667423970014267246553110800667267853616970529812738203125516169205531952973978205310
print(factorint(enc).keys())
prime_factors = factorint(enc).keys()

dict_keys([2, 5, 7, 11, 13, 17, 23, 43, 59, 61, 67, 73, 79, 83, 103, 109, 113, 127, 139, 149, 163, 167, 179, 181, 191, 193, 197, 211, 227, 229, 233, 251, 257, 271, 277, 281, 293, 307, 313, 337, 349, 353, 367, 373, 383, 397, 401, 419, 421, 443, 449, 467, 487, 491, 541, 557, 563, 571, 577, 587, 593, 599, 607, 619, 631, 641, 647, 653, 691, 701, 727, 743, 757, 761, 797, 811, 821, 823, 829, 839, 863, 877, 887, 907, 911, 929, 937, 947, 967, 977, 983, 991, 1013, 1019, 1033, 1039, 1051, 1061, 1069, 1087, 1091, 1093, 1103, 1109, 1117, 1151, 1153, 1171, 1213, 1217, 1237, 1249, 1277, 1279, 1291, 1297, 1307, 1319])


In [6]:
indices = [i for i, e in enumerate(primes) if e in prime_factors]
print(indices)

[0, 2, 3, 4, 5, 6, 8, 13, 16, 17, 18, 20, 21, 22, 26, 28, 29, 30, 33, 34, 37, 38, 40, 41, 42, 43, 44, 46, 48, 49, 50, 53, 54, 57, 58, 59, 61, 62, 64, 67, 69, 70, 72, 73, 75, 77, 78, 80, 81, 85, 86, 90, 92, 93, 99, 101, 102, 104, 105, 106, 107, 108, 110, 113, 114, 115, 117, 118, 124, 125, 128, 131, 133, 134, 138, 140, 141, 142, 144, 145, 149, 150, 153, 154, 155, 157, 158, 160, 162, 164, 165, 166, 169, 170, 173, 174, 176, 177, 179, 180, 181, 182, 184, 185, 186, 189, 190, 192, 197, 198, 202, 203, 205, 206, 209, 210, 213, 214]


In [7]:
from Cryptodome.Util.number import long_to_bytes

flag_long = sum(map(lambda x: 2**x, indices))
dec_flag = long_to_bytes(flag_long)
print(dec_flag)

b'flag{functi0n_h4cking_ftw!}'
